# Market Making with Order Book Imbalance

Imbalance: $I_t=\frac{\sum b_t - \sum a_t}{\sum b_t + \sum a_t}$. Quotes:
$Q^{bid}_t = m_t - \frac{s_t}{2}(1+\kappa I_t) - \phi\, inv_{t-1}$,
$Q^{ask}_t = m_t + \frac{s_t}{2}(1-\kappa I_t) + \phi\, inv_{t-1}$.

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
np.random.seed(0)
T=3000
mid=100+np.cumsum(0.0+0.1*np.random.randn(T))
spread=0.04+0.02*np.abs(np.random.randn(T))
L=10
bids=np.stack([mid - (i+1)*spread for i in range(L)],axis=1)
asks=np.stack([mid + (i+1)*spread for i in range(L)],axis=1)
size=lambda i: 2.0*np.exp(-0.2*i)
bid_sz=np.stack([size(i)*np.ones(T) for i in range(L)],axis=1)
ask_sz=np.stack([size(i)*np.ones(T) for i in range(L)],axis=1)
B=bid_sz.sum(1); A=ask_sz.sum(1)
imb=(B-A)/(B+A+1e-9)


## Quoting with inventory control

In [ ]:
kappa=0.6; phi=0.01
q_bid=mid - 0.5*2*spread*(1+kappa*imb)
q_ask=mid + 0.5*2*spread*(1-kappa*imb)
inv=np.zeros(T); cash=0.0; pnl=np.zeros(T)


## Depth-aware simulator

In [ ]:
def fills(qb,qa,bids_t,asks_t,bsz_t,asz_t,imb):
    lam=0.2
    take_buy = np.random.rand()<lam*(1+max(0,-imb))*0.5
    take_sell= np.random.rand()<lam*(1+max(0, imb))*0.5
    fb=fa=0.0; cash=0.0
    if qb>=bids_t[0]: fb+=min(0.3*bsz_t[0],1.0); cash-=fb*qb
    if qa<=asks_t[0]: fa+=min(0.3*asz_t[0],1.0); cash+=fa*qa
    if take_buy and qa<=asks_t[0]: q=0.5; fa+=q; cash+=q*qa
    if take_sell and qb>=bids_t[0]: q=0.5; fb+=q; cash-=q*qb
    return fb,fa,cash


## Simulation

In [ ]:
for t in range(T):
    qb=q_bid[t]-phi*(inv[t-1] if t>0 else 0.0)
    qa=q_ask[t]+phi*(inv[t-1] if t>0 else 0.0)
    fb,fa,dc=fills(qb,qa,bids[t],asks[t],bid_sz[t],ask_sz[t],imb[t])
    inv[t]=(inv[t-1] if t>0 else 0.0)+fb-fa
    cash+=dc; pnl[t]=cash+inv[t]*mid[t]


## Results

In [ ]:
ret=np.diff(pnl,prepend=pnl[0])
sh=np.mean(ret)/np.std(ret+1e-9)*np.sqrt(252*6*60)
print('Final PnL',pnl[-1],'Sharpe~',sh)
plt.figure(); plt.plot(pnl); plt.title('PnL');
plt.figure(); plt.plot(inv); plt.title('Inventory'); plt.show()
